## Setup

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain-openai langchain-community langchainhub langgraph langchain_huggingface
%pip install --upgrade --quiet transformers accelerate

### Conectar con BBDD

In [2]:
from langchain_community.utilities import SQLDatabase

usuario = 'postgres'
password = 'place_rag_password'
host = 'localhost'     # o la IP/URL de tu servidor
puerto = '5432'        # puerto por defecto de PostgreSQL
base_datos = 'place_rag_db'

# Crear la URL de conexión
uri = f"postgresql+psycopg2://{usuario}:{password}@{host}:{puerto}/{base_datos}"

db = SQLDatabase.from_uri(uri)

print(db.dialect)                  # Para verificar el tipo de base de datos
print(db.get_usable_table_names()) # Para listar las tablas de la BD
db.run("SELECT * FROM documentos LIMIT 30;")  # Ejemplo de consulta

postgresql
['documentos', 'entidades', 'expedientes', 'paises', 'regiones']


"[('PCAP 50 equipos trabajo en movilidad Anexo I acuerdo MP.pdf', 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=0b72a403-f89f-44fe-852d-dcba825cdef4', 'Pliego Administrativo', '2023/20'), ('PPT Suministro 50 portatiles Anexo II acuerdo MP.pdf', 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=91b4b1e6-5e64-46e7-8091-27febb59c5fd', 'Pliego Técnico', '2023/20'), ('04 PCAP 35 2023 FIRMADO.pdf', 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=8bed1139-4bab-4327-9849-9d3f9ed091f3', 'Pliego Administrativo', 'CON 35/2023 SE AB'), ('03 PPT GRUPOS DE BOMBEO 2023 FIRMADA.pdf', 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&sour

## Chains

### Application State

In [2]:
from typing_extensions import TypedDict

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

### Create SQL query from text

In [19]:
from dotenv import load_dotenv
load_dotenv()
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=0)

In [20]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate([
    ("system", "Dada una pregunta de entrada, crea una consulta de {dialect} sintácticamente correcta para ejecutarla y ayudar a encontrar la respuesta. A menos que el usuario especifique en su pregunta una cantidad específica de ejemplos que desea obtener, limita siempre la consulta a como máximo {top_k} resultados. Puedes ordenar los resultados por una columna relevante para devolver los ejemplos más interesantes de la base de datos. Nunca consultes todas las columnas de una tabla específica, solo solicita algunas columnas relevantes según la pregunta. Presta atención a utilizar solo los nombres de las columnas que puedes ver en la descripción del esquema. Ten cuidado de no consultar columnas que no existen. Además, presta atención a qué columna está en cada tabla. Utiliza únicamente las siguientes tablas: {table_info} Pregunta {input_question}")
])

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
from typing_extensions import Annotated

class QueryOutput(TypedDict):
    """Generated SQL query."""
    query: Annotated[str, ..., "Syntactically valid SQL query."]

llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = prompt_template.invoke(
        {
            "dialect": "postgresql",
            "top_k": "10",
            "table_info": "expedientes, documentos, entidades, paises, regiones",
            "input_question": state["question"]
        })
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

structured_llm = llm.with_structured_output(QueryOutput)

In [ ]:
hf.invoke("Hello, write a python function for saying hello to the Antarctica")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Hello, write a python function for saying hello to the Antarctica.\n\n### Input\n\nThe input'

## Ejecutar la query

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

## Generar respuesta

In [ ]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

### --- Test query ---

In [22]:
write_query({"question": "¿Cuantos documentos hay en la base de datos?"})

{'query': 'SELECT COUNT(*) AS total_documentos FROM documentos;'}

### Vector Store

In [3]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


entidades = query_as_list(db, "SELECT name FROM entidades")
tipo_contrato = query_as_list(db, "SELECT procurement_project_type_code FROM expedientes")
subtipo_contrato = query_as_list(db, "SELECT procurement_project_subtype_name FROM expedientes")
estado_expediente = query_as_list(db, "SELECT contract_folder_status_code FROM expedientes")
paises = query_as_list(db, "SELECT country_name FROM paises")
regiones = query_as_list(db, "SELECT country_subentity_name FROM regiones")

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [7]:
from langchain.agents.agent_toolkits import create_retriever_tool

_ = vector_store.add_texts(entidades + tipo_contrato + subtipo_contrato + estado_expediente + paises + regiones)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

### --- Test retriever ---

In [10]:
print(retriever_tool.invoke("Sumninitros"))

Suministros

Yugoiztochen

Montserrat

Severoiztochen

Soria


: 